### --- Day 11: Monkey in the Middle ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/11

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 11);
WriteLines(inputLines, maxRows: 13);

Loading puzzle file: Day11.txt
Total lines: 55
Max line length: 48

Monkey 0:
  Starting items: 66, 71, 94
  Operation: new = old * 5
  Test: divisible by 3
    If true: throw to monkey 7
    If false: throw to monkey 4

Monkey 1:
  Starting items: 70
  Operation: new = old + 6
  Test: divisible by 17
    If true: throw to monkey 3
    If false: throw to monkey 0


In [4]:
string[] testInputLines = 
[
    "Monkey 0:",
    "  Starting items: 79, 98",
    "  Operation: new = old * 19",
    "  Test: divisible by 23",
    "    If true: throw to monkey 2",
    "    If false: throw to monkey 3",
    "",
    "Monkey 1:",
    "  Starting items: 54, 65, 75, 74",
    "  Operation: new = old + 6",
    "  Test: divisible by 19",
    "    If true: throw to monkey 2",
    "    If false: throw to monkey 0",
    "",
    "Monkey 2:",
    "  Starting items: 79, 60, 97",
    "  Operation: new = old * old",
    "  Test: divisible by 13",
    "    If true: throw to monkey 1",
    "    If false: throw to monkey 3",
    "",
    "Monkey 3:",
    "  Starting items: 74",
    "  Operation: new = old + 3",
    "  Test: divisible by 17",
    "    If true: throw to monkey 0",
    "    If false: throw to monkey 1",
];

In [5]:
// Firstly, let's parse monkey definitions:

// Monkey 0:
//   Starting items: 79, 98
//   Operation: new = old * 19
//   Test: divisible by 23
//     If true: throw to monkey 2
//     If false: throw to monkey 3

using Operation = (char op, string value);

class Monkey
{
    public int Id { get; init; }
    public Queue<Item> Items { get; init; }
    public Operation Operation { get; init; }
    public int DivisibleTest { get; init; }
    public int TrueRecipient { get; init; }
    public int FalseRecipient { get; init; }
    public int InspectionCount { get; set; }
}

class Item(long worryLevel) 
{
    public long WorryLevel { get; set; } = worryLevel;
}

int[] Numbers(string inputLine) => inputLine.ParseAll(@"\d+").Select(int.Parse).ToArray();

Monkey ParseMonkey(string[] inputLines)
{
    Monkey monkey = new()
    {
        Id = Numbers(inputLines[0])[0],
        Items = new(Numbers(inputLines[1]).Select(i => new Item(i))),
        Operation = (inputLines[2][23], inputLines[2][25..]),
        DivisibleTest = Numbers(inputLines[3])[0],
        TrueRecipient = Numbers(inputLines[4])[0],
        FalseRecipient = Numbers(inputLines[5])[0],
    };

    return monkey;
}

In [6]:
class MonkeyGame
{
    public List<Monkey> Monkeys { get; init; }

    public Func<long, long> WorryResolver { get; set; } = w => w / 3;

    public void DoMonkeyRound()
    {
        foreach (var i in Enumerable.Range(0, Monkeys.Count))
        {
            DoOneMonkey(i);
        }
    }

    void DoOneMonkey(int monkeyId)
    {
        var monkey = Monkeys[monkeyId];

        while (monkey.Items.TryDequeue(out var item))
        {
            monkey.InspectionCount++;

            var oldWorry = item.WorryLevel;
            var newWorry = monkey.Operation switch 
            {
                ('*', "old") => oldWorry * oldWorry,
                ('*', var x) => oldWorry * long.Parse(x),
                ('+', "old") => oldWorry + oldWorry,
                ('+', var x) => oldWorry + long.Parse(x),
                _ => throw new Exception("Unexpected operation")
            };
            item.WorryLevel = WorryResolver(newWorry);

            var nextMonkey = (item.WorryLevel % monkey.DivisibleTest) switch 
            {
                0 => monkey.TrueRecipient,
                _ => monkey.FalseRecipient
            };
            Monkeys[nextMonkey].Items.Enqueue(item);
        }
   }
}

MonkeyGame ParseMonkeyGame(string[] inputLines)
{
    var monkeys = inputLines.SeparateBy(line => line is "")
                    .Select(ParseMonkey)
                    .OrderBy(m => m.Id)
                    .ToList();
    return new MonkeyGame { Monkeys = monkeys };
}

long RunMonkeySim(string[] inputLines, int rounds = 20, Func<long, long> worryResolver = null)
{
    var monkeyGame = ParseMonkeyGame(inputLines);
    
    // Spoiler alert: this changes in part 2
    var wr = worryResolver ?? monkeyGame.WorryResolver;
    monkeyGame.WorryResolver = wr;

    foreach (var _ in Enumerable.Range(0, rounds))
    {
        monkeyGame.DoMonkeyRound();
    }

    var (a, b) = monkeyGame.Monkeys.Select(m => m.InspectionCount)
                    .OrderByDescending(m => m)
                    .ToArray();
    
    long result = a;
    result *= b;
    return result;
}

In [7]:
// In this example, the two most active monkeys inspected items 101 and 105
// times. The level of monkey business in this situation can be found by
// multiplying these together: 10605.

var testAnswer = RunMonkeySim(testInputLines);
Console.WriteLine(testAnswer);

10605


In [8]:
// Figure out which monkeys to chase by counting how many items they inspect
// over 20 rounds. What is the level of monkey business after 20 rounds of
// stuff-slinging simian shenanigans?

var part1Answer = RunMonkeySim(inputLines);
Console.WriteLine(part1Answer);

55944


In [9]:
// 55944 is correct!
Ensure(55944, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/11

So after some messing around, I noticed the multiplications performed by the monkey operations will overflow 64-bit integers. I also noticed the divisible tests are all prime numbers! Therefore I think we can truncate the large number into the lowest common multiple of these primes, and the divisible tests will still have the same result. 

In [11]:
// This number is the product of all the divisible numbers for the testInput
// monkeys:
Func<long, long> testWR = a => a % 96577;

// After 10000 rounds, the two most active monkeys inspected items 52166 and
// 52013 times. Multiplying these together, the level of monkey business in this
// situation is now 2713310158.

var part2TestAnswer = RunMonkeySim(testInputLines, rounds: 10_000, worryResolver: testWR);
Console.WriteLine(part2TestAnswer);

2713310158


In [12]:
// This number is the product of all the divisible numbers for the puzzle input
// monkeys:
Func<long, long> part2WR = a => a % 9699690;

// Starting again from the initial state in your puzzle input, what is the level
// of monkey business after 10000 rounds?

var part2Answer = RunMonkeySim(inputLines, rounds: 10_000, worryResolver: part2WR);
Console.WriteLine(part2Answer);

15117269860


In [13]:
// 15117269860 is correct!
Ensure(15117269860L, part2Answer);